In [10]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import keras
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pathlib
import seaborn as sns
import tensorflow as tf
#import tensorflow_addons as tfa

from keras.utils import plot_model
from imblearn.over_sampling import SMOTE
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import BatchNormalization, Conv2D, Dense, Dropout, Flatten, GlobalAveragePooling2D, Input, MaxPool2D
from keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator
#from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
#from tensorflow.keras.applications import ResNet50V2, VGG16, InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
folder = '/content/drive/MyDrive/datasetModified/BreaKHis_v1/histology_slides/breast'
folder_path = pathlib.Path(folder)

In [5]:
train_generator = ImageDataGenerator(rescale = 1./255, brightness_range=[0.8, 1.2], zoom_range=[.99, 1.01],
                                     data_format="channels_last", fill_mode="constant", horizontal_flip=True)
train_data_gen = train_generator.flow_from_directory(directory=folder, target_size=(128,128), batch_size=6500, shuffle=False)

Found 2081 images belonging to 2 classes.


In [6]:
train_data, train_labels = train_data_gen.next()

In [7]:
print(train_data.shape, train_labels.shape)

(2081, 128, 128, 3) (2081, 2)


In [8]:
sm = SMOTE(random_state=42)

train_data, train_labels = sm.fit_resample(train_data.reshape(-1, 128 * 128 * 3), train_labels)

print(train_data.shape, train_labels.shape)

(2874, 49152) (2874, 1)


In [9]:
train_data = train_data.reshape(-1, 128, 128, 3)
print(train_data.shape, train_labels.shape)

(2874, 128, 128, 3) (2874, 1)


In [11]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_labels,test_labels = train_test_split(train_data, train_labels, test_size = 0.2, random_state=42)

train_data, val_data, train_labels,val_labels = train_test_split(train_data, train_labels, test_size = 0.2, random_state=42)

In [12]:
val_data.shape

(460, 128, 128, 3)

In [13]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=(128,128,3)))

model.add(Conv2D(filters=64, kernel_size=(5,5), padding='same', activation='softmax'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.5))
model.add(Flatten())

model.add(Dense(32, activation='softmax'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [14]:
from tensorflow.keras.optimizers import RMSprop,SGD,Adam

opt1 = Adam(lr = 0.001)
opt2 = RMSprop(lr = 0.001)
opt3 = SGD(lr = 0.001)

In [16]:
#plot_model(model)

In [17]:
#model.summary()

In [ ]:

# model.compile(optimizer=opt1, loss='binary_crossentropy', metrics=['accuracy'])

# history = model.fit(train_data,train_labels,
#                     validation_data=(val_data,val_labels),
#                     steps_per_epoch=80,
#                     epochs=100,
#                     validation_steps=50,
#                     verbose=1)

Epoch 1/100
80/80 [==============================] - ETA: 0s - loss: 0.6946 - accuracy: 0.5057

80/80 [==============================] - 19s 79ms/step - loss: 0.6946 - accuracy: 0.5057 - val_loss: 0.6932 - val_accuracy: 0.4913
Epoch 2/100
80/80 [==============================] - 4s 49ms/step - loss: 0.6934 - accuracy: 0.5073
Epoch 3/100
80/80 [==============================] - 4s 51ms/step - loss: 0.6935 - accuracy: 0.5041
Epoch 4/100
80/80 [==============================] - 4s 50ms/step - loss: 0.6942 - accuracy: 0.4807
Epoch 5/100
80/80 [==============================] - 4s 50ms/step - loss: 0.6934 - accuracy: 0.5073
Epoch 6/100
80/80 [==============================] - 4s 50ms/step - loss: 0.6940 - accuracy: 0.4715
Epoch 7/100
80/80 [==============================] - 4s 53ms/step - loss: 0.6929 - accuracy: 0.5024
Epoch 8/100
80/80 [==============================] - 4s 50ms/step - loss: 0.6935 - accuracy: 0.4899
Epoch 9/100
80/80 [==============================] - 4s 50ms/step - loss: 0.6929 - accuracy: 0.5155
Epoch 10/100
80/80 [==============================] - 4s 52ms/step - 

In [18]:
# Adding BatchNormalization Layers to check if it affects loss value

model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=(128,128,3)))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=(5,5), padding='same', activation='softmax'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(32, activation='softmax'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [19]:
# SGD optimizer performance for learning rate 0.001

model.compile(optimizer=opt3, loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(train_data,train_labels,
                    validation_data=(val_data,val_labels),
                    steps_per_epoch=80,
                    epochs=100,
                    validation_steps=50,
                    verbose=1,
                   callbacks = EarlyStopping(monitor='val_loss'))

Epoch 1/100
80/80 [==============================] - ETA: 0s - loss: 0.6484 - accuracy: 0.6166

80/80 [==============================] - 20s 94ms/step - loss: 0.6484 - accuracy: 0.6166 - val_loss: 0.6931 - val_accuracy: 0.4913
Epoch 2/100
80/80 [==============================] - ETA: 0s - loss: 0.6246 - accuracy: 0.6444

80/80 [==============================] - 5s 64ms/step - loss: 0.6246 - accuracy: 0.6444
Epoch 3/100
80/80 [==============================] - ETA: 0s - loss: 0.6154 - accuracy: 0.6422

80/80 [==============================] - 5s 67ms/step - loss: 0.6154 - accuracy: 0.6422
Epoch 4/100
80/80 [==============================] - ETA: 0s - loss: 0.6136 - accuracy: 0.6438

80/80 [==============================] - 5s 66ms/step - loss: 0.6136 - accuracy: 0.6438
Epoch 5/100
80/80 [==============================] - ETA: 0s - loss: 0.6006 - accuracy: 0.6498

...

80/80 [==============================] - 5s 67ms/step - loss: 0.3203 - accuracy: 0.8537
Epoch 97/100
80/80 [==============================] - ETA: 0s - loss: 0.2976 - accuracy: 0.8744

80/80 [==============================] - 5s 68ms/step - loss: 0.2976 - accuracy: 0.8744
Epoch 98/100
80/80 [==============================] - ETA: 0s - loss: 0.3086 - accuracy: 0.8706

80/80 [==============================] - 5s 66ms/step - loss: 0.3086 - accuracy: 0.8706
Epoch 99/100
80/80 [==============================] - ETA: 0s - loss: 0.2920 - accuracy: 0.8880

80/80 [==============================] - 5s 67ms/step - loss: 0.2920 - accuracy: 0.8880
Epoch 100/100
80/80 [==============================] - ETA: 0s - loss: 0.2795 - accuracy: 0.8940

80/80 [==============================] - 5s 66ms/step - loss: 0.2795 - accuracy: 0.8940


In [21]:
# cnn_loss_history = history.history['loss']
# cnn_loss_val_history = history.history['val_loss']
# epoch_count = range(1, len(cnn_loss_history)+1)
# plt.plot(epoch_count,cnn_loss_history,'r-')
# plt.plot(epoch_count,cnn_loss_val_history ,'b-')
# plt.legend(['Train loss','Val loss'])
# plt.xlabel('epoch')
# plt.ylabel('Loss')
# plt.show()

In [22]:
pred = model.predict(test_data)

18/18 [==============================] - 2s 55ms/step


In [23]:
pred

array([[0.87056696],
       [0.95073146],
       [0.95073384],
       [0.91499496],
       [0.9506372 ],
       [0.9507068 ],
       [0.81208223],
       [0.95073354],
       [0.94950384],
       [0.12697496],
       [0.1128078 ],
       [0.80018675],
       [0.95071507],
       [0.12396529],
       [0.11778466],
       [0.9493253 ],
       [0.9507293 ],
       [0.9490723 ],
       [0.9506981 ],
       [0.94044703],
       [0.9507073 ],
       [0.9013182 ],
       [0.03955634],
       [0.95073396],
       [0.5156614 ],
       [0.07858887],
       [0.08314081],
       [0.2599284 ],
       [0.12382112],
       [0.9507325 ],
       [0.94863707],
       [0.12052875],
       [0.9477884 ],
       [0.0822152 ],
       [0.14147292],
       [0.05410914],
       [0.13840157],
       [0.08465049],
       [0.95073384],
       [0.9507336 ],
       [0.948902  ],
       [0.9507333 ],
       [0.94398004],
       [0.95072824],
       [0.9501434 ],
       [0.95071626],
       [0.13208014],
       [0.950

In [24]:
# Assuming you have your predicted values stored in the variable 'predictions'
# Assuming you have your actual labels stored in the variable 'test_labels'

# Convert the predicted probabilities to binary values (0 or 1) based on a threshold
binary_predictions = (pred > 0.5).astype(int)

# Calculate the accuracy by comparing the binary predictions with the actual labels
accuracy = (binary_predictions == test_labels).mean()

# Print the accuracy
print("Accuracy:", accuracy)

Accuracy: 0.8904347826086957


In [26]:
model.save("model.keras")

In [27]:
loaded_model = tf.keras.saving.load_model("model.keras")

In [29]:
model.save('/content/1/',
                save_format='tf', save_traces=False)